In [1]:
import os
import torch
from os.path import dirname,splitext,basename,exists
from PIL import Image
import cv2
# from yolov5 import detect



In [40]:
# function detect videos
def detect_video(
video_path,# video path
file_name, #file name
save_dir = None # save path
):
    list_of_original_frames = []
    list_of_detected_classes_in_all_frames = []
    detected_classes = []
    check_dict = {}
    # i for naming each detected frame
    i=0

    print(video_path)
    cap = cv2.VideoCapture(video_path)
    if (cap.isOpened() is False):
        print("Error opening the video file")
        # Read fps and frame count
    else:
        # video info
        # use 5 or CAP_PROP_FPS
        fps = cap.get(cv2.CAP_PROP_FPS)        
        print('Frames per second : ', fps,'FPS')
        # You can replace 7 with CAP_PROP_FRAME_COUNT as well, they are enumerations
        frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        print('Frame count : ', frame_count)
        while (cap.isOpened()):
            ret, frame = cap.read()
            if ret == False:
                break
            
            # cv2.imshow('Frame',frame)
            frame = frame[..., ::-1] # refine the frames from OpenCV image (BGR to RGB)
            
            list_of_original_frames.append(frame)
            # 20 is in milliseconds, try to increase the value, say 50 and observe
            # key = cv2.waitKey(20)
            # if key == ord('q'):
            #     break
            # else:
            #     break
        cap.release()
        cv2.destroyAllWindows()

    for file in list_of_original_frames:
        print('image {} out of {}'.format(i,frame_count))
        results = model(file)
        results.print()
        print(type(results.pred))
        tensor = torch.stack(results.pred)
        # print(type(tensor))
        print(tensor)
        empty_tensor = torch.empty(1,0,6)

        print(empty_tensor)
    
        # equate if they are the same it will be true otherwise false
        is_empty = torch.equal(tensor,empty_tensor)
        # if the tensor is not empty meaning their was prediction save otherwise dont meaning false
        if not is_empty:  
            name = file_name+'_frame_'+str(i)
            path = save_dir+name
            results.save(path)
              # image_results = cv2.imwrite(file_name+str(i)+'.jpg',results)
        i+=1

        new_df = results.pandas()
        new_result = new_df.xywh[0]['name']
        list_of_detected_classes_in_all_frames.append(new_result.tolist())
   
    
    for outer_loop in list_of_detected_classes_in_all_frames:
        for inner_loop in outer_loop:
            if inner_loop:
                detected_classes.append(inner_loop)

    for iterate in detected_classes:
        if iterate not in check_dict:
            check_dict[iterate] = 1
        else:
            check_dict[iterate] = check_dict[iterate] + 1
    
    return check_dict

In [57]:
def predict(
  data_path, #data path
  weight_path = None, # weight path eg. './yolov5/weight/best.pt'
  model_conf = 0.3,  # model confidence
  save_dir= 'static/detect', # detect path 
  webcam = False, # webcam
  size = 640 # default 416
):
  # model setup
  # load model in its own environment loading locally for now


  model = torch.hub.load('./yolov5', 'custom', source='local', path=weight_path ) # for PIL/cv2/np inputs and  
  # confidences
  model.conf = model_conf

  # use cpu instead of cuda 
  model.cpu()
  # data to be infered is stores hre
  data =[]
  # batch of images stored here
  imgs =[]
  # videos url stores here
  vids =[]
  # append data path to the data array
  data.append(data_path)

  vids_save_dir= save_dir +'/vids/'
  imgs_save_dir=  save_dir +'/img/'

  for f in data:
    if exists(f):
      file_basename = basename(f)
      file_ext = splitext(file_basename)[1]
      file_name = splitext(file_basename)[0]
      print(file_ext)
      if file_ext in ['.jpg', '.png'] and webcam is False:
        # img1 = Image.open(f)  # PIL image
        img = cv2.imread(f)[..., ::-1]  # OpenCV image (BGR to RGB)
        imgs.append(img)
        # Inference
        results = model(imgs, size=size)  # includes NMS


        # print and save results
        results.print()  
        results.save(imgs_save_dir)  
        # results.show()
      elif file_ext in ['.mp4', '.avi'] and webcam is False:
        #  read the video file
        # append all the videos to be inferenced
        vids.append(f)
        # Inference
        for vid in vids:
          check_dict=detect_video(vid,file_name,vids_save_dir)
        # print and save results
        check_dict.values
        print(check_dict) 
        # check_dict.save('static/detect/vids')  
      elif webcam is True:
        print('do nothing')
        !python detect.py --weights weight_path  --conf model_conf --source 0 --project ./static/detect/
    else:
      error_file_path = "file doesn\'t exist"
      print(error_file_path)       




In [59]:


# image size
# --img 416

!python ./yolov5/detect.py --weights ./yolov5/weight/best.pt --conf 0.35 --source ./static/uploads/vids/VID_20211111_204321.mp4 --project ./static/detect/vids --view-img --save-txt

Parse error at "'install '": Expected stringEnd


detect: weights=['./yolov5/weight/best.pt'], source=./static/uploads/vids/VID_20211111_204321.mp4, imgsz=[640, 640], conf_thres=0.35, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=./static/detect/vids, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  2021-11-10 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients
video 1/1 (1/104) C:\Users\edwin\Documents\WasteClassfication\DrinkWasteWebDemo\static\uploads\vids\VID_20211111_204321.mp4: 640x384 Done. (0.920s)
video 1/1 (2/104) C:\Users\edwin\Documents\WasteClassfication\DrinkWasteWebDemo\static\uploads\vids\VID_20211111_204321.mp4: 640x384 Done. (0.856s)
video 1/1 (3/104) C:\Users\edwin\Documents\WasteClassfication\DrinkWasteWebDemo\static\uploads\vids\VID_20211111_204321.mp4

In [43]:
data_example= ['./yolov5/data/images','./static/uploads/img/IMG_20211107_194950.jpg','./static/uploads/vids/VID_20211111_204321.mp4']

In [56]:
weights = './yolov5/weight/best.pt'
for d in data_example:
 predict(d,weights,model_conf=0.35)

YOLOv5  2021-11-10 torch 1.10.0+cpu CPU

Fusing layers... 


Parse error at "'install '": Expected stringEnd


Model Summary: 232 layers, 7254609 parameters, 0 gradients
Adding AutoShape... 
YOLOv5  2021-11-10 torch 1.10.0+cpu CPU

Fusing layers... 



Parse error at "'install '": Expected stringEnd


Model Summary: 232 layers, 7254609 parameters, 0 gradients
Adding AutoShape... 


...

C:\Users\edwin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
image 1/1: 3120x4160 2 PETs
Speed: 96.6ms pre-process, 448.6ms inference, 3.0ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to static\detect\img
YOLOv5  2021-11-10 torch 1.10.0+cpu CPU

Fusing layers... 


Parse error at "'install '": Expected stringEnd


Model Summary: 232 layers, 7254609 parameters, 0 gradients
Adding AutoShape... 


.mp4


UnidentifiedImageError: cannot identify image file './static/uploads/vids/VID_20211111_204321.mp4'

In [1]:
!python ./yolov5/detect.py --conf 0.35 --source ./static/uploads/vids/VID_20211111_204321.mp4 --project ./static/detect/vids --view-img --save-txt

Parse error at "'install '": Expected stringEnd


detect: weights=yolov5\yolov5s.pt, source=./static/uploads/vids/VID_20211111_204321.mp4, imgsz=[640, 640], conf_thres=0.35, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=./static/detect/vids, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  2021-11-10 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/104) C:\Users\edwin\Documents\WasteClassfication\DrinkWasteWebDemo\static\uploads\vids\VID_20211111_204321.mp4: 640x384 1 zebra, Done. (4.275s)
video 1/1 (2/104) C:\Users\edwin\Documents\WasteClassfication\DrinkWasteWebDemo\static\uploads\vids\VID_20211111_204321.mp4: 640x384 1 zebra, Done. (1.034s)
video 1/1 (3/104) C:\Users\edwin\Documents\WasteClassfication\DrinkWasteWebDemo\static\uploads\vids\VID_20211111_20

In [5]:
r = []
data = [{3,'ew'},{2,'we'},{1,'e'}]
n = 4
for i in range(0,n):
  for j in data:
     print(j)
  r.append(j)


{3, 'ew'}
{2, 'we'}
{1, 'e'}
{3, 'ew'}
{2, 'we'}
{1, 'e'}
{3, 'ew'}
{2, 'we'}
{1, 'e'}
{3, 'ew'}
{2, 'we'}
{1, 'e'}


In [6]:
print(r)

[{1, 'e'}, {1, 'e'}, {1, 'e'}, {1, 'e'}]
